---
title: "Cubic Zirconia Price Analysis"
format: gfm
jupyter: python3
---


# Introduction

The goal of this analysis is to explore the relationships between cubic zirconia gemstone characteristics (e.g., `carat`, `cut`, `color`, `clarity`) and price. The analysis emphasizes inference, focusing on identifying significant predictors and understanding their relationships while ensuring statistical rigor.

------------------------------------------------------------------------


```{r}
# Load required libraries
library(tidyverse)
library(car) 
library(dplyr)
library(MASS) 
library(gridExtra)
```


# Load the dataset


```{r}
path <- "cubic_zirconia.csv"

data <- read_csv(file = path, col_names = TRUE)
head(data)
```


Response Variable:

```         
Price : the Price of the cubic zirconia.
```

Possible Covariates:

```         
Carat : Carat weight of the cubic zirconia.

Cut : Describe the cut quality of the cubic zirconia. Quality is increasing order Fair, Good, Very Good, Premium, Ideal.

Color: Colour of the cubic zirconia.With D being the best and J the worst.

Clarity : cubic zirconia Clarity refers to the absence of the Inclusions and Blemishes. (In order from Best to Worst, FL = flawless, I3= level 3 inclusions) - FL, IF, VVS1, VVS2, VS1, VS2, SI1, SI2, I1, I2, I3

Depth : The Height of a cubic zirconia, measured from the Culet to the table, divided by its average Girdle Diameter.

Table : The Width of the cubic zirconia's Table expressed as a percentage of its Average Diameter.

X : Length of the cubic zirconia in mm.

Y : Width of the cubic zirconia in mm.

Z : Height of the cubic zirconia in mm.
```

# Clean the dataset


```{r}
#Removing row count column
data <- data[-1]
head(data)
```


# Exploratory Data Analysis (EDA)

## Summary Statistics


```{r}
print(str_glue ("There are {nrow(data)} rows in the dataset.\n\n"))
print(str_glue("There are {sum(is.na(data))} total missing values in the dataset.\n Total missing values per column:\n\n"))

print(colSums(is.na(data)))
data <- data |> drop_na(depth)
print(colSums(is.na(data)))
```


# Correcting Column Data Types


```{r}

data$cut <- factor(data$cut, levels = c("Ideal", "Premium", "Very Good", "Good", "Fair"))
data$color <- factor(data$color, levels = c("D", "E", "F", "G", "H", "I", "J"))
data$clarity <- factor(data$clarity, levels = c("FL", "IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"))

head(data)

```


## Distributions of Variables


```{r}
# Function to calculate proportions and create a plot
create_proportion_plot <- function(var, var_name) {
  prop_table <- prop.table(table(var)) * 100
  prop_df <- as.data.frame(prop_table)
  colnames(prop_df) <- c(var_name, "proportion")
  
  ggplot(prop_df, aes(x = get(var_name), y = proportion)) +
    geom_bar(stat = "identity", fill = "purple", color = "white") +
    labs(x = var_name, y = "Proportion (%)", title = str_glue("Proportion of Categories in {str_to_title(var_name)}")) +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1),
         text = element_text(size = 15)) 
}

p1 <- create_proportion_plot(data$clarity, "clarity")
p2 <- create_proportion_plot(data$cut, "cut")
p3 <- create_proportion_plot(data$color, "color")

grid.arrange(p1, p2, p3, nrow = 1, ncol = 3, widths = c(8, 8, 8), heights = c(8))
```


The graphs above indicate that the categories within each categorical variable are not equally represented in the data.


```{r}
ggplot(data, aes(x = price)) + 
  geom_histogram(bins = 15, fill = "purple", color = "white") +
  labs(x = "Price ($)", y = "Count", title = "Price Distribution") + 
  theme(text = element_text(size = 16)) +
  geom_vline(aes(xintercept = mean(price), color = "Mean"), linetype = "dashed", size = 1) + 
  geom_vline(aes(xintercept = median(price), color = "Median"), linetype = "dashed", size = 1) + 
  scale_color_manual(values = c("Mean" = "red", "Median" = "blue")) +
  theme(legend.title = element_text(size = 14), legend.text = element_text(size = 12))

```


The above histogram for the response variable, Price, shows a right skewed distribution. The mean is higher than the median.

## Relationships Between Variables


```{r}
# graphing all continuous variables against price (the response)

continuous_vars = c('carat', 'depth', 'table', 'x', 'y', 'z')
labels = c('Carat', 'Depth', 'Table (%)', 'Length (mm)', 'Width (mm)', 'Height (mm)')

plot_list <- list()

for (i in 1:length(continuous_vars)) {
    p <- ggplot(data = data, aes(x = .data[[continuous_vars[i]]], y = price)) +
            geom_point(color = "purple", alpha = 0.2) + 
            labs(x = labels[i], y = "Price ($)", title = str_glue("Price vs {stringr::str_to_title(continuous_vars[i])}")) + 
            theme(text = element_text(size = 12))
    plot_list[[i]] <- p
}

grid.arrange(grobs = plot_list, nrow = 3, col = 2)
```

todo

```{r}
cat_vars = c('cut', 'color', 'clarity')

for (i in 1:length(cat_vars)) {
    p <- ggplot(data = data, aes(x = .data[[cat_vars[i]]], y = price, color = .data[[cat_vars[i]]])) +
            geom_boxplot(alpha = 0.5) + 
            labs(x = stringr::str_to_title(cat_vars[i]), y = "Price ($)", title = str_glue("Price vs {stringr::str_to_title(cat_vars[i])}")) + 
            theme(text = element_text(size = 15))
    print(p)
}
```


There appears to be a lot of overlap between boxplots in the cut and color plots. This suggests there might not be a strong relationship between different cut and color categories and price. However, there seems to be less overlap between boxes in the clarity plot. For example, there is almost no overlap between the IF (internally flawless) and I1 (level 1 inclusions) categories. This indicates that clarity may be a stronger predictor of price. This is intuitive because gemstones with higher clarity are generally more expensive. Cut and color may not be as strongly correlated with price.

It is interesting to note that in the third plot of price vs clarity, the median price for IF gemstones is lower than the median price for I1 gemstones. This however, is counterintuitive because a flawless gemstone should have the highest price.

The following graph suggests that the lower price for IF gemstones may have been because of lower weights of such cubic zirconia being bought. This suggests there may be some interaction between weight of the gemstone and its clarity in determining the price of a gemstone.


```{r}
ggplot(data = data, aes(x = clarity, y = carat, color = clarity)) +
            geom_boxplot(alpha = 0.5) + 
            labs(x = "Clarity", y = "Weight (carat)", title = "Carat vs Clarity") + 
            theme(text = element_text(size = 15))
```

```{r}
ggplot(data, aes(x = carat, y = price)) + geom_point(alpha = 0.5) + geom_smooth(method = "lm", col = "red") + labs(title = "Price vs Carat")
```

Based on the graph above, we assume that the model to be linear (i.e we do not include higher degree term in our model). There is a slight curve in the graph, so we have decided to include the interaction between carat and clarity to introduce some non-linearity.

# Multicollinearity Analysis

## Variance Inflation Factor (VIF)


```{r}
model_initial <- lm(price ~ carat + color + cut + clarity + depth + table + x + y + z, data = data)

vif_values <- vif(model_initial)
print(vif_values)
```


-   `carat`: **22.79** → Severe multicollinearity.

-   `x`, `z`: **92.71** and **78.46** → Extremely severe multicollinearity.

-   `y`: **14.53** → Moderate to severe multicollinearity.

-   `depth`: **2.30** and `table`: **1.14** → These are within acceptable ranges.

We saw that the VIF for 'x', 'y', 'z' are high. However, 'x', 'y', 'z' measures the dimension of the diamond and one would think these variables to be independent. Therefore, we created a new variable, namely 'volume' to address the problem of multicollinearity.


```{r}
data$volume <- data$x * data$y * data$z

model_with_volume <- lm(price ~ carat + cut + color + clarity + depth + table + volume, data = data)
vif_with_volume <- vif(model_with_volume)
print("VIF for Model with Volume:")
print(vif_with_volume)


model_without_volume <- lm(price ~ carat + cut + color + clarity + depth + table, data = data)
vif_without_volume <- vif(model_without_volume)
print("VIF for Model without Volume:")
print(vif_without_volume)

```


From the above output, we can see that VIF's for carat and volume are above the general threshold of 5, suggesting multicollinearity. 

1. We can drop 'volume' since it is intuitive to think that 'carat' (weight) and 'volume' are related. Additionally, 'carat' seems more important of a measure for gemstones, as it is more commonly used in practice to measure gemstones. Keeping only `carat` simplifies the model and avoids redundancy. This is the ideal approach if we wish to retain the interpretability of the model.

2. Another thing we can do is we can try using Principle Component Regression to address the multicollinearity between carat and volume. This is a good approach if we only wish to create a high accuracy prediction model and not necessarily retain interpretability.

# Outlier Detection

From the exploratory data analysis, we can see the presence of a lot of outliers. Here, we will address the problem of outliers.

## Standardized Residuals


```{r}
# Standardized residuals
model <- lm(price ~ carat + cut + clarity + depth + table, data = data)
data$residuals <- rstandard(model)

# Identify outliers
outliers <- data %>% filter(abs(residuals) > 3)  # Residuals > 3 standard deviations
print(nrow(outliers))
print(outliers)

```

We can see that we have 586 outliers. This is 586/26967 * 100 ~ 2.17% of our data. Since we are not sure if these outliers are the result of data entry errors, etc, we will not remove the outliers as that may lead to deletion of useful information. 

# Regression Analysis After Dropping Volume

## Stepwise Selection


```{r}
predictors <- c("carat", "cut", "color", "clarity", "depth", "table") 
response <- "price"
```


# Fit initial and full models


```{r}
simple_model <- lm(as.formula(paste(response, "~ 1")), data = data)
full_model <- lm(price ~ carat * clarity + cut + color + depth + table, data = data)

stepwise_model <- stepAIC(simple_model, scope = list(lower = simple_model, upper = full_model), direction = "both")
summary(stepwise_model)
```

Based on the EDA and due to computational cost, we only include 1 interaction term, namely 'carat:clarity' and assuming no interaction between the other covariates.

```{r}
par(mfrow = c(2, 2)) 
plot(stepwise_model)
```

The Residuals vs Fitted plot does not show homoscedasticity. The Q-Q plot also suggests that the residuals do not really follow a normal distribution. Therefore, we need to address by applying log transform on the response variable (price) and one of the covariates (carat).
# Results and Interpretation


```{r}
# Calculate key metrics
rss <- sum(resid(stepwise_model)^2)
tss <- sum((data$price - mean(data$price))^2)
r_squared <- 1 - (rss / tss)
adjusted_r_squared <- 1 - ((1 - r_squared) * ((nrow(data) - 1) / (nrow(data) - length(coef(stepwise_model)) - 1)))

# Create a summary table
library(tibble)
model_metrics <- tibble(
  Metric = c("R²", "Adjusted R²"),
  Value = c(r_squared, adjusted_r_squared)
)
print(model_metrics)

```


# Log Transformation


```{r}
data$log_price <- log(data$price) 
data$log_carat <- log(data$carat)

# performing backward selection
log_transformed_model <- lm(log_price ~ log_carat + cut + color + clarity + table + depth + log_carat:clarity, data = data)
summary(log_transformed_model)

# excluding table
log_transformed_model <- lm(log_price ~ log_carat + cut + color + clarity + depth + log_carat:clarity, data = data)
summary(log_transformed_model)

# excluding depth
log_transformed_model <-lm(log_price ~ log_carat + cut + color + clarity + log_carat:clarity, data = data)
summary(log_transformed_model)
```

Based on the summary above, after applying log transformation on 'carat' and dropping 'depth', and 'table' we achieved better R^2 and adjusted R^2. This simpler model is easier for interpretation and has the same adjusted R^2 as the larger models. Since the interaction between some clarity levels and carat are significant, we have chosen to include it in the model. 


```{r}
par(mfrow = c(2, 2)) 
plot(log_transformed_model)
```

Now, the residuals vs fitted plot suggests heteroscedasticity is less of a concern as the points are now centering around zero and have constant variance. However, another possible concern is that of a slight curve in the residuals, which might suggest a non-linear relationship. We have already included one non linear relationship - that of the interaction term, however it may not be enough.

The Q-Q plot looks better than the previous model's, although the deviations at the end suggest a distribution with heavier tails rather than a normal distribution. Alternatively, it may also suggest the presence of outliers, which is present as analyzed previously. 


```{r}
# Trying a cubic relationship between log(carat) and log(price)

cubic_relation <- lm(log_price ~ log_carat + I(log_carat^2) + I(log_carat^3) + cut + color + clarity + 
    log_carat:clarity, data = data)

summary(cubic_relation)

plot(y = cubic_relation$residuals, x = cubic_relation$fitted.values, main = "Residual Plot")
```


Although the above cubic model has a higher adjusted R^2 : 0.9849 vs the old model :  0.9831, the additional complexity of the cubic model is not justified by the small increase of 0.0018 in R^2. The residual plot also doesn't look much better than the previous model. So even though the cubic log(carat) value is significant in the summary, we have decided to choose the more simplistic model as we have already captured some non-linearity with our interaction term between clarity and log_carat.

Our final model for this section is $ log(price) = \beta_0 + \beta_1*log(carat) + \beta_2*cut + \beta_3*color + \beta_4*clarity + \beta_5*log(carat)*clarity $


# Regression Analysis Using PCA To Address Multicollinearity

We will now compare the above method of dropping the volume variable with the PCA method.


```{r}
# Extracting carat and volume covariates into their own dataframe:

data <- read_csv(file = path, col_names = TRUE) |> drop_na(depth)
data$volume <- data$x * data$y * data$z

subset_data <- data |> dplyr::select(volume, carat)
head(subset_data)
```

```{r}
# Applying PCA on standardized carat and volume

pc_car_vol <- princomp(subset_data, cor = T)
plot(pc_car_vol)

```

The above graph shows that most of the variance is captured by the first component. The code cell below calculates that around 98% of variance is captured by the first component. Therefore, we can now regress our model on only the first component as the second isn't needed given that it is only capturing around 2% of variation.


```{r}
lambdas <- pc_car_vol$sdev^2

# Computing variation proportion captured by first X principal components
( sum(lambdas[1]) / sum(lambdas) )  # First PC
( sum(lambdas[1:2]) / sum(lambdas) )  # First two PCs
```

```{r}
pcr_model <- lm(log(price) ~ (pc_car_vol$scores[,1]) + I(pc_car_vol$scores[,1]^2) + I(pc_car_vol$scores[,1]^3) + cut + color + clarity + clarity:pc_car_vol$scores[,1] + depth + table, data = data)
summary(pcr_model)

plot(y = pcr_model$residuals, x = pcr_model$fitted.values, main = "Residual Plot")
qqnorm(pcr_model$residuals, pch = 1, frame = FALSE)
qqline(pcr_model$residuals, col = "steelblue", lwd = 2)
```

The residual plot and qq-plot show similar concerns as the model fitted before. Namely, the qq plot's curve at the end suggests the presence of outliers or heavier tailed distribution of the residuals rather than a normal distribution. The residual plot also suggests the presence of some outliers. There is not as much concern for heteroscedasticity.

As seen above, the pca model required a cubic first principal component and two more covariates: namely depth and table, to be approximately the same adjusted R^2 as the previous method's model. The previous model's adjusted R^2 was 0.9831 and the pcr model's adjusted R^2 is 0.9787. This is a difference of 0.0044, which is rather insignificant. This suggests that while both method perform similarly in explaining the variation in the data, the first model (created by dropping the volume covariate), is better as it is simpler to interpret. This choice is in line with the principle of parsimony. Therefore, we conclude the better model for this data is the model created by dropping the volume variable:

$ log(price) = \beta_0 + \beta_1*log(carat) + \beta_2*cut + \beta_3*color + \beta_4*clarity + \beta_5*log(carat)*clarity $
